In [131]:
!pip install pipreqs
!pipreqs . --force

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 317.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 449.9 kB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=497e86d97bb245685b122b630b8c97a5ae7231cf54f62d4d22e02c214da54339
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.12.3 which is incompatible.


INFO: Not scanning for jupyter notebooks.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in ./requirements.txt


In [ ]:
## 1. Installation and Setup

In [ ]:
!curl ipv4.icanhazip.com

34.46.47.233


In [ ]:
!cp /content/drive/MyDrive/clinical_RAG/discharge_notes_40_patients.csv /content/

In [ ]:
%%writefile app.py
import streamlit as st
import os
import faiss
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from together import Together
from typing import List, Dict, Tuple
import re

Overwriting app.py


In [ ]:
%%writefile -a app.py
os.environ["TOGETHER_API_KEY"] = st.secrets["TOGETHER_API_KEY"]
os.environ["HUGGINGFACE_HUB_TOKEN"] = st.secrets["HUGGINGFACE_HUB_TOKEN"]
from huggingface_hub import login
login(token=os.environ["HUGGINGFACE_HUB_TOKEN"])

Appending to app.py


## 2. Data Loading and Preprocessing

In [ ]:
%%writefile -a app.py
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import io
from googleapiclient.http import MediaIoBaseDownload

# Auth
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["GDRIVE_CREDENTIALS"],  # loaded from secrets.toml
    scopes=SCOPES
)

# Build service
service = build('drive', 'v3', credentials=credentials)

# File ID
file_id = '1JJBW3fE8GZLVZQgPt1CS7HSYWeB2uqE8-p9xtIReIxU'

# Download file
request = service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while not done:
    _, done = downloader.next_chunk()

fh.seek(0)
df = pd.read_csv(fh)

Appending to app.py


In [ ]:
%%writefile -a app.py
# config
BIOBERT_MODEL = "emilyalsentzer/Bio_ClinicalBERT"
EMBED_DIM = 768  # BioBERT output dim
# load models
tokenizer_biobert = AutoTokenizer.from_pretrained(BIOBERT_MODEL)
biobert = AutoModel.from_pretrained(BIOBERT_MODEL)

Appending to app.py


# 3. ClinicalBERT Embedding Generation

In [ ]:
%%writefile -a app.py
def get_embedding(text: str) -> np.ndarray:
    inputs = tokenizer_biobert(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = biobert(**inputs)
    attention_mask = inputs['attention_mask']
    last_hidden = outputs.last_hidden_state
    mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
    masked_output = last_hidden * mask
    mean_pooled = masked_output.sum(1) / mask.sum(1)
    return mean_pooled.squeeze().cpu().numpy()


Appending to app.py


# 4. FAISS Index Building

In [ ]:
%%writefile -a app.py

def build_faiss_index(embeddings: List[np.ndarray]) -> faiss.IndexFlatL2:  # Change to faiss.IndexFlat
    """
    Builds a FAISS index using the provided embeddings.

    Args:
        embeddings: A list of NumPy arrays representing the embeddings.

    Returns:
        A FAISS index object (faiss.IndexFlat).
    """
    dimension = embeddings[0].shape[0]  # Get dimensionality from embeddings
    index = faiss.IndexFlat(dimension)  # Change to faiss.IndexFlat
    index.add(np.vstack(embeddings))  # Add embeddings to the index
    return index

Appending to app.py


## 5. Text Chunking
Using Note Sections/Headers

In [ ]:
%%writefile -a app.py
def chunk_text(input_text: str, charttime, max_chars: int = 500) -> List[Tuple[str, str]]:
    """
    Chunks a clinical text into smaller pieces based on sections and a maximum
    character limit.

    Args:
        input_text: The clinical text to be chunked.
        max_chars: The maximum number of characters allowed in each sub-chunk.

    Returns:
        A list of (section, chunk text, date) tuples.
    """

    section_headers = [
        "Service:", "Allergies:",
        "Chief Complaint:", "Major Surgical or Invasive Procedure:", "History of Present Illness:",
        "Past Medical History:", "Social History:", "Family History:", "Physical Exam:",
        "PHYSICAL EXAM ON ADMISSION:", "PHYSICAL EXAM ON DISCHARGE:", "Pertinent Results:",
        "Brief Hospital Course:", "Medications on Admission:", "Discharge Medications:",
        "Discharge Disposition:", "Facility:", "Discharge Diagnosis:", "Discharge Condition:",
        "Discharge Instructions:", "Followup Instructions:"
    ]

    pattern = re.compile(rf"^({'|'.join(map(re.escape, section_headers))})", re.MULTILINE)
    matches = list(pattern.finditer(input_text))

    chunks = []
    for i in range(len(matches)):
        start = matches[i].start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(input_text)
        header = matches[i].group(1)
        content = input_text[start:end].strip()

        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', content)
        current_chunk = ""
        for sentence in sentences:
            if len(current_chunk) + len(sentence) <= max_chars:
                current_chunk += sentence
            else:
                chunks.append((header, current_chunk.strip(), charttime))
                current_chunk = sentence
    chunks.append((header, current_chunk.strip(), charttime))

    return chunks

Appending to app.py


chunk structure:
```
all_chunks = {
  # patient  : list of their note chunks + metadata
  subject_id : [(section, chunk text, time)]
  
}
```

## 7. Querying with FAISS

In [ ]:
%%writefile -a app.py
# Constants
SCORE_THRESHOLD = 50   # Adjust based on empirical observation
MAX_CHUNKS = 15

def search_chunks_with_faiss(all_chunks, subject_id, query, top_k=30):
    """
    Searches FAISS and filters chunks by score and count.
    Assumes all_chunks[subject_id] = list of tuples (text, date).
    """
    all_texts_with_dates = all_chunks.get(subject_id, [])
    if not all_texts_with_dates:
        print(f"No chunks found for subject ID: {subject_id}")
        return []

    # Build index using only the chunk text part of the tuples
    all_texts = [text for section, text, date in all_texts_with_dates]

    # embed all chunks
    embeddings = [get_embedding(text) for text in all_texts]
    index = build_faiss_index(embeddings) # build FAISS index from embeddings

    # Search
    query_emb = get_embedding(query) #embed query and use to search
    D, I = index.search(query_emb.reshape(1, -1), top_k)

    # Filter by score and cap to max chunks, include date
    results = []
    for i, score in zip(I[0], D[0]):
        if score <= SCORE_THRESHOLD:
            # Get section, text, and date from the original list using i
            section, text, date = all_texts_with_dates[i]
            results.append((text, section, date, score))
            if len(results) >= MAX_CHUNKS:
                break
    return results


Appending to app.py


## 8. Response Generation with LLaMA
 + display in streamlit app

In [ ]:
%%writefile -a app.py
os.environ["TOGETHER_API_KEY"] = os.environ["TOGETHER_API_KEY"]
client = Together(api_key=os.environ["TOGETHER_API_KEY"])
MODEL_NAME = "meta-llama/Llama-4-Scout-17B-16E-Instruct"

Appending to app.py


In [ ]:
%%writefile -a app.py
def generate_response_from_chunks(query: str, retrieved_chunks: List[tuple]) -> str:
    """
    Generates a clinically relevant, faithful, and date-aware summary using retrieved context.
    """
    # Format each chunk as a date-tagged clinical note
    context = "\n".join([f"[{section} {date}] {text.strip()}" for text, section, date, score in retrieved_chunks])

    # Optimized prompt
    prompt = f"""You are a clinical assistant helping summarize a patient's medical history for a physician during clinical assessment.

Patient Timeline (each entry includes a section, date, and note):
{context}

Query: "{query}"

Instructions:
- Extract only relevant and factual information from the timeline.
- Summarize findings related to the query in clinical terms.
- Associate each finding with its date.
- Do not hallucinate or infer conditions that are not explicitly mentioned.
- Present the response in a clear, concise manner suitable for use in clinical decision-making.

Answer:"""
    # Print the length of the prompt
    print("== PROMPT:==")
    print(prompt)

    # Print the length of the prompt
    print(f"Prompt length: {len(prompt)}")

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

Appending to app.py


In [ ]:
%%writefile -a app.py

# Streamlit app structure
st.set_page_config(page_title="Clinical Chatbot", layout="centered")
st.title("🩺 Clinical Chatbot Assistant")

# Subject ID input (fixed or from dropdown in future)
subject_id_to_search = st.number_input("Patient Subject ID", value=10001217)

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# Initialize chunk dictionary only once
if "all_chunks" not in st.session_state:
    all_chunks = {}
    for index, row in df.iterrows():
        subject_id = row['subject_id']
        if subject_id == subject_id_to_search:
            note = row['text']
            charttime = row['charttime']
            chunks = chunk_text(note, charttime)
            all_chunks[subject_id] = chunks
    st.session_state.all_chunks = all_chunks

# Display chat history
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Chat input box
user_query = st.chat_input("Enter your clinical question...")

if user_query:
    # Display user message
    st.session_state.messages.append({"role": "user", "content": user_query})
    with st.chat_message("user"):
        st.markdown(user_query)

    # Process user query
    with st.spinner("Searching notes and generating response..."):
        retrieved_chunks = search_chunks_with_faiss(
            st.session_state.all_chunks,
            subject_id_to_search,
            user_query
        )
        response = generate_response_from_chunks(user_query, retrieved_chunks)

    # Display assistant response
    st.session_state.messages.append({"role": "assistant", "content": response})
    with st.chat_message("assistant"):
        st.markdown(response)

Appending to app.py


In [ ]:
!npm install localtunnel
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧⠙your url is: https://calm-oranges-send.loca.lt
^C


Testing Inputs:
- Password: 34.46.47.233

- Query: Does the patient show signs of neurological issues like face numbness?